In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# LOAD DATA
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

y_train = keras.utils.to_categorical(y_train.astype(np.int32), 10 )
y_test = keras.utils.to_categorical(y_test.astype(np.int32) , 10)

In [9]:
# вспомогательная функция для отображения изображений
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

In [46]:
codings_size = 30
# Define G
generator = keras.Sequential([
    keras.layers.Dense(units=100, activation='elu', input_shape=[codings_size]),
    keras.layers.Dense(units=150, activation='elu'),
    keras.layers.Dense(units=784, activation='sigmoid'),
    keras.layers.Reshape([28, 28])
])

In [47]:
# Define D
discriminator = keras.Sequential([
    keras.layers.Flatten(input_shape=[28 , 28]),
    keras.layers.Dense(units=150, activation='elu'),
    keras.layers.Dense(units=100, activation='elu'),
    keras.layers.Dense(units=1, activation='sigmoid')
])

In [48]:
# на вход генератора подается "посевное" распределение размерности coding_size
# на выходе - результат, считает ли дискриминатор полученное (от генератора)
# изображение настоящим или нет
gan = keras.models.Sequential([generator, discriminator])

In [49]:
# собираем дискриминатор как бинарный классификатор
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [50]:
# собираем обычную состязательную сеть
gan.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [51]:
batch_size = 32

dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [58]:
"""
Здесь определяем функции для тренировки состязательной сети 
"""
def train(gan, dataset, n_epochs=5, batch_size=30, codings_size=30):
    generator, discriminator = gan.layers
    
    for ep in range(n_epochs):
        
        print("epoch #{} / {}".format(ep+1, n_epochs))
    
        for X_batch in dataset:
            # train Discriminator
            discriminator.trainable = True
            generator.trainable = False
            # генерируем нормальный шум размерности (batch size, codings size)
            noise = tf.random.normal(shape=[batch_size, codings_size])
            # получаем сгенерированные "картинки", представляющие собой просто шум
            generated = generator(noise)
            # чтобы помочь генератору научиться воссоздавать нужные нам объекты,
            # конкатенируем шумовое изображение и реальное
            X_fake_and_real = tf.concat([generated, X_batch], axis=0)
            # ?
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            # обучаем дискриминатор именно на этом пакете объектов (batch)
            discriminator.train_on_batch(X_fake_and_real, y1)
            
            # train Generator
            generator.trainable = True
            discriminator.trainable = False
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)

            gan.train_on_batch(noise, y2)
            
        plot_multiple_images(generated, 6)
        plt.show()


In [60]:
%%time

train(gan, dataset, n_epochs = 3, batch_size=32, codings_size=100)

epoch #1 / 3


InvalidArgumentError: Matrix size-incompatible: In[0]: [32,100], In[1]: [30,100] [Op:MatMul]

In [ ]:
, c